In [ ]:
import embed
from embed.demos import usc

In [ ]:
# What we want to do: 
#  Save serializatinos of sections that are embeddeable

    
{   
    section.attrib['identifier']: text for section in usc.get_direct_sections(root)
    if embed.count_tokens(text := usc.serialize_xml_clean(section)) <= embed.TOKEN_LIMIT
}


In [ ]:
# identifier, status, text
# identifier, status, text, embedding